In [6]:
import rasterio
from rasterio.enums import Resampling
from rasterio.warp import reproject, calculate_default_transform

# Listar archivos raster
raster_files = [
    r"F:\Servicio INIFAP\MaizEdo_Actual\Resultados\TempEdoMexClass.tif",
    r"F:\Servicio INIFAP\MaizEdo_Actual\Resultados\AlturasClass.tif",
    r"F:\Servicio INIFAP\MaizEdo_Actual\Resultados\PrecipClass.tif",
    r"F:\Servicio INIFAP\MaizEdo_Actual\Resultados\SuelosClass.tif",
    r"F:\Servicio INIFAP\MaizEdo_Actual\Resultados\FasesfyqClass.tif",
    r"F:\Servicio INIFAP\MaizEdo_Actual\Resultados\TexturasClass.tif",
    r"F:\Servicio INIFAP\MaizEdo_Actual\Resultados\PendientesClass.tif",
]
#Listar pesos en el orden correspondiente a la lista de rasters
  
weights = [0.24, 0.22, 0.20,0.19,0.08,0.05,0.02]  


# Determinar el tamaño y la transformación comunes para el remuestreo
with rasterio.open(raster_files[0]) as src0:
    dst_crs = src0.crs
    dst_transform, dst_width, dst_height = calculate_default_transform(
        src0.crs, dst_crs, src0.width, src0.height, *src0.bounds)

# Inicializar un array para la suma ponderada
weighted_sum = np.zeros((dst_height, dst_width), dtype=np.float32)

# Procesar cada raster
for raster_file, weight in zip(raster_files, weights):
    with rasterio.open(raster_file) as src:
        # Leer los datos y realizar el remuestreo
        data = src.read(1)
        reprojected_data = np.empty((dst_height, dst_width), dtype=np.float32)

        reproject(
            source=data,
            destination=reprojected_data,
            src_transform=src.transform,
            src_crs=src.crs,
            dst_transform=dst_transform,
            dst_crs=dst_crs,
            resampling=Resampling.nearest  # o cualquier otro método de resampling adecuado
        )
        
        # Realizar la suma ponderada
        weighted_sum += reprojected_data * weight

# Guardar el resultado en un nuevo archivo raster
with rasterio.open(raster_files[0]) as src0:
    out_meta = src0.meta.copy()
    out_meta.update({
        "driver": "GTiff",
        "height": dst_height,
        "width": dst_width,
        "transform": dst_transform,
        "crs": dst_crs
    })

with rasterio.open(r'F:\Servicio INIFAP\MaizEdo_Actual\archivo_resultado.tif', 'w', **out_meta) as dest:
    dest.write(weighted_sum, 1)